In [2]:
import pandas as pd

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

In [7]:
from dotenv import load_dotenv

load_dotenv()

# import os

# os.environ["OPENAI_API_KEY"] = "..."

True

# 1. LLM 기반 질의 생성

이번 실습에서는 QA 데이터 셋을 만들어보겠습니다.

먼저 LLM 기반 질의 생성을 통해 15개의 QA 데이터 셋을 만들어보겠습니다.

####  Corpus 파일 불러오기

In [5]:
corpus_dir = os.path.join(root_dir, 'autorag_project', 'final', 'chunk', '0.parquet')
corpus = pd.read_parquet(corpus_dir)
corpus

,doc_id,contents,path,start_end_idx,metadata
0,c7036095-887a-4082-885e-14209c68cb0f,<h1 id='0' style='font-size:18px'>2024년 KBO 규약...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 575]","{'last_modified_datetime': '2025-01-27', 'next..."
1,fc96bdca-c580-4f27-ab43-dfbcdbda99fb,고용할 수 있음</td></tr><tr><td>제44조 [선수계약의 승인]</td>...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[546, 1071]","{'last_modified_datetime': '2025-01-27', 'next..."
2,18e02344-dc14-4924-b38c-9573bd437fcb,<h1 id='0' style='font-size:20px'>제4장 연 고 권</h...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 619]","{'last_modified_datetime': '2025-01-27', 'next..."
3,fd7ba604-3428-4b9b-8dea-8565b394cb34,히어로즈</td><td>서울특별시</td></tr><tr><td>NC 다이노스</t...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[621, 1277]","{'last_modified_datetime': '2025-01-27', 'next..."
4,ef1e83f5-7a4d-419b-b469-f4c4eb53b101,<table id='0' style='font-size:14px'><tr><td>구...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 516]","{'last_modified_datetime': '2025-01-27', 'next..."
...,...,...,...,...,...
856,bb36c5db-a016-404f-ad42-2e2c1eab11e4,그림 8. 정식경기 \r\n<예1>\r\n이닝 1 2 3 4 5 6 7 8 9 \r...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 438]","{'last_modified_datetime': '2025-01-27', 'next..."
857,167952f2-5c16-461c-b139-f7b7d8cf9f40,갑 1 0 0 1 0 0 1 |3\r\n을 0 0 1 0 0 0 0+∝ |1,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[416, 455]","{'last_modified_datetime': '2025-01-27', 'next..."
858,621dcf2e-3fdd-4b8e-8514-b94353524da3,그림 9. 정식경기\r\n<예8>\r\n이닝 1 2 3 4 5 6 7 8 9 \r\...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 447]","{'last_modified_datetime': '2025-01-27', 'next..."
859,006c3812-75b3-4d24-a703-fa261cd59417,3-2 승 \r\n<예14>\r\n이닝 1 2 3 4 5 6 7 8 9 \r\n갑 ...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[429, 526]","{'last_modified_datetime': '2025-01-27', 'next..."


#### Run QA Creation

In [11]:
from autorag.data.qa.schema import Corpus
from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import (
    make_basic_gen_gt,
    make_concise_gen_gt,
)
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

llm = OpenAI()  # model = "gpt-3.5-turbo"
initial_corpus = Corpus(corpus)

In [12]:
initial_qa = (
    initial_corpus.sample(random_single_hop, n=15) # sample only three
    .map(
        lambda df: df.reset_index(drop=True),
    )
    .make_retrieval_gt_contents()
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
		lang='ko',
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,
		lang='ko',
    )
    .batch_apply(
        make_concise_gen_gt,  # answer generation (concise)
        llm=llm,
		lang='ko',
    )
    .filter(
        dontknow_filter_rule_based,  # filter don't know
        lang="ko",
    )
)

[01/30/25 13:41:57] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=104390;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=54340;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=407218;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=39631;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=113183;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=876790;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=821425;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=554018;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=52032;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=479813;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=566952;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=230724;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=945617;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=727288;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 13:42:05] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=71336;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=468101;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=596446;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=389840;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 13:42:08] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=126298;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=378286;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

#### 결과 확인

In [13]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,6d1ec768-e872-44fe-9080-6a4b8828da09,[[2ea156c5-99b4-4338-9306-d139abe8b44e]],[[5.09 아웃\r\n․ 58 ․\r\n다 빨리 2루에 도달했고 타자주자는 1루에...,포스 상태가 해제되는 경우는 무엇입니까?,"[포스 상태가 해제되는 경우는 태그 플레이가 발생할 때입니다., 타자가 주자가 됨에..."
1,f99ba0f3-b2da-4cdb-98fc-380fbb2f5e17,[[a389cfdf-361a-45b7-bf77-91ab137e8392]],"[[본 규칙 5.02(c)의 적용을 모면하거나 회피하려고 시도한다면, \r\n심판은...",규칙 5.02(c)를 어떻게 모면하거나 회피하려고 시도하는 경우 어떤 벌칙이 부과되나요?,"[규칙 5.02(c)를 모면하거나 회피하려고 시도하는 경우, 수비팀이 해당 규칙을 ..."
2,f3afe648-ccdd-414b-9dd7-8c054c031bc3,[[2bed192d-9591-405c-a39f-025caee6a7e4]],[[다음 연도 신인 드래프트에서 동일 라운드 종료 후 추가 보\r\n상 지명을 실시...,KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO ...,[KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO...
3,50aa50bd-ef0d-47a8-bd18-0b8c7eddd253,[[3d329be3-89db-4e3b-bb83-6ac1d6eb23e6]],"[[출장정지 제재를 받은 자는 제재 종료일까지 KBO 정규시즌, \r\n포스트시즌,...",출장정지 제재를 받은 자는 어떤 경기에 출장할 수 없나요?,"[출장정지 제재를 받은 자는 KBO 정규시즌, 포스트시즌, 퓨처스리그 경기에 출장할..."
4,d6fd53eb-42ee-4835-83eb-72afac55bf51,[[96db994a-b5f5-49c4-8fff-e6668148da3f]],[[치수를 잴 때는 계\r\n측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉...,치수를 잴 때 어디에 계측기나 줄자를 접촉시켜야 하나요?,"[계측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉시켜야 합니다., 글러브의..."
5,0bcf8d62-9497-4f5a-a4fa-8626b9b5018b,[[4662da6c-ae20-4889-8289-0fd53b00cc76]],[[쳤다. 땅볼을 받은 \r\n1루수는 1루주자를 포스 아웃시키려고 2루에 송구하였...,"1루수가 1루주자를 포스 아웃시키려고 2루에 송구했을 때, 심판이 투수의 주루방해를...","[심판이 투수의 주루방해를 인정하면 ""업스트럭션""을 선고하고 볼을 데드로 하며, 이..."
6,edb95b03-e5aa-4ce7-9307-08505e3c377f,[[07090a61-45c2-4280-b2fb-b3e4647f0e88]],"[[고의낙구 하더라도 5.09⒜⑿의 규정에 관계없이 볼 인 플레이며, \r\n인필드...",인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 어떤...,[인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 인...
7,9e337798-8cb6-4e56-8ebe-a7196abd3888,[[6ebf10d4-1abe-4d0c-b0fb-cc5c0b77eb6d]],"[[시상 \r\n투수 부문 - 평균자책점상, 승리상\r\n타자 부문 - 타율상, 홈...","야구 시상에서 투수 부문과 타자 부문으로 나누어 시상하는데, 어떤 상이 시상되는가?","[야구 시상에서 투수 부문은 평균자책점상과 승리상이 시상되며, 타자 부문은 타율상,..."
8,6a16546d-d5cf-491c-abd7-0df5ff9a5c22,[[3d0914ba-78e8-4572-b4bb-fc6695c94130]],[[설\r\n사 관중이 그 플레이를 방해하더라도 수비방해의 이득을 얻을 수는 없\r...,야구에서 덕아웃 부근에서 플라이 볼을 잡으려는 야수가 덕아웃 안으로 빠지지 않으려고...,"[공을 잡으면 정규의 포구가 됩니다., 정규의 포구]"
9,a43cac50-414e-4d66-80ce-3c882dfa1bd9,[[494d83d0-761c-4e71-ae71-de73106e9bda]],[[플레이 시도 시 슬라이딩\r\n주자가 더블 플레이 성립을 방해하기 위한 목적으로...,야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'이란 무엇입니까?,[야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'은 베이스...


결과를 보니 질문이 잘 나온 것도 있지만 2번째 질문처럼 이상한 질문이 나온 것도 있습니다.

2번 질문 같이 문서를 노골적으로 보고 말하는 거 같은 질문 (규칙 5.02(c)를 ~)을 만들지 않기 위해, 
프롬프트를 Custom으로 제작해봐야겠습니다.

좀 더 정확한 평가를 위해서는 2번 질문 같은 질문은 지워주지만, 이번 실습에서는 포함 시키도록 하겠습니다 : )

일단 만들어둔 qa 데이터셋을 잘 저장해줍니다. (저장은 필수입니다 !!)

In [16]:
llm_qa_path = os.path.join(root_dir, 'autorag_project', 'final', 'qa', 'llm_qa.parquet')
initial_qa.data.to_parquet(llm_qa_path)

# 2. Human in the Loop 실습

Custom 프롬프트를 함께 작성해봅시다!

In [18]:
before_prompt = """
당신은 스타트업의 지원 프로그램 중 하나인 "글로벌 팁스 프로그램"에 대해 알려주는 어시스턴트의 관리자입니다.
RAG 시스템을 관리를 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 글로벌 팁스 프로그램에 대한 안내 문서입니다.
문서를 보고 예상되는 사용자들의 질문을 적으세요.
단, 반드시 하나의 질문만을 생성하세요.
생성하는 질문에는 미사여구 없이, 해당 질문만이 포함되어야 합니다.
"""

In [19]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from autorag.data.qa.query.llama_gen_query import llama_index_generate_base
from llama_index.core.base.llms.base import BaseLLM
from typing import Dict

prompt = """
당신은 대한민국의 야구 리그인 KBO의 규정, 규칙, 규약에 대해서 대답하는 시스템의 관리자입니다.
KBO에 관한 질문에 대답해주는 시스템을 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 2024 KBO리그 규정, 규칙과 규약에 관한 문서입니다.
문서의 내용만을 이용해서 예상되는 사용자들의 질문을 적으세요.
문서의 구체적인 규칙 내용까지는 질문에서 언급하지 마시오.
단, 반드시 하나의 질문만을 생성하세요.
"""

async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content=prompt,
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [20]:
second_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/30/25 14:03:13] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=40253;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=983223;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=365568;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=275946;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=478866;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=956289;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=798072;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=320613;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=407430;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=953221;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=522027;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=199014;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=189368;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=600847;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=854479;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=792676;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=169456;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=145662;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 14:03:16] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=66785;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=396965;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

#### 결과 확인

In [22]:
second_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,6d1ec768-e872-44fe-9080-6a4b8828da09,[[2ea156c5-99b4-4338-9306-d139abe8b44e]],[[5.09 아웃\r\n․ 58 ․\r\n다 빨리 2루에 도달했고 타자주자는 1루에...,포스 플레이가 아닌 태그 플레이에서 주자가 제3아웃되기 전에 본루에 들어간 주자의 ...,"[포스 상태가 해제되는 경우는 태그 플레이가 발생할 때입니다., 타자가 주자가 됨에..."
1,f99ba0f3-b2da-4cdb-98fc-380fbb2f5e17,[[a389cfdf-361a-45b7-bf77-91ab137e8392]],"[[본 규칙 5.02(c)의 적용을 모면하거나 회피하려고 시도한다면, \r\n심판은...",규칙 5.02(c)를 회피하거나 모면하려는 행위에 대해 어떤 벌칙이 부과되나요?,"[규칙 5.02(c)를 모면하거나 회피하려고 시도하는 경우, 수비팀이 해당 규칙을 ..."
2,f3afe648-ccdd-414b-9dd7-8c054c031bc3,[[2bed192d-9591-405c-a39f-025caee6a7e4]],[[다음 연도 신인 드래프트에서 동일 라운드 종료 후 추가 보\r\n상 지명을 실시...,어떤 경우에만 신인 드래프트 참가 신청서를 제출한 후에도 KBO 소속 구단과 선수계...,[KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO...
3,50aa50bd-ef0d-47a8-bd18-0b8c7eddd253,[[3d329be3-89db-4e3b-bb83-6ac1d6eb23e6]],"[[출장정지 제재를 받은 자는 제재 종료일까지 KBO 정규시즌, \r\n포스트시즌,...",어떤 경우에 출장정지 제재를 받게 되나요?,"[출장정지 제재를 받은 자는 KBO 정규시즌, 포스트시즌, 퓨처스리그 경기에 출장할..."
4,d6fd53eb-42ee-4835-83eb-72afac55bf51,[[96db994a-b5f5-49c4-8fff-e6668148da3f]],[[치수를 잴 때는 계\r\n측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉...,치수를 잴 때 어떤 위치에 계측기나 줄자를 접촉시켜야 하나요?,"[계측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉시켜야 합니다., 글러브의..."
5,0bcf8d62-9497-4f5a-a4fa-8626b9b5018b,[[4662da6c-ae20-4889-8289-0fd53b00cc76]],[[쳤다. 땅볼을 받은 \r\n1루수는 1루주자를 포스 아웃시키려고 2루에 송구하였...,"업스트럭션 상황에서 2루에서의 포스 아웃이 먼저 이루어졌을 때, 어떤 상황이 발생하나요?","[심판이 투수의 주루방해를 인정하면 ""업스트럭션""을 선고하고 볼을 데드로 하며, 이..."
6,edb95b03-e5aa-4ce7-9307-08505e3c377f,[[07090a61-45c2-4280-b2fb-b3e4647f0e88]],"[[고의낙구 하더라도 5.09⒜⑿의 규정에 관계없이 볼 인 플레이며, \r\n인필드...",인필드 플라이 규칙이 우선되는 상황에서 수비방해가 선언되면 어떤 결정이 내려지나요?,[인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 인...
7,9e337798-8cb6-4e56-8ebe-a7196abd3888,[[6ebf10d4-1abe-4d0c-b0fb-cc5c0b77eb6d]],"[[시상 \r\n투수 부문 - 평균자책점상, 승리상\r\n타자 부문 - 타율상, 홈...",KBO 리그에서 시상을 받는 선수들은 어떤 부문으로 나누어 선정되나요?,"[야구 시상에서 투수 부문은 평균자책점상과 승리상이 시상되며, 타자 부문은 타율상,..."
8,6a16546d-d5cf-491c-abd7-0df5ff9a5c22,[[3d0914ba-78e8-4572-b4bb-fc6695c94130]],[[설\r\n사 관중이 그 플레이를 방해하더라도 수비방해의 이득을 얻을 수는 없\r...,야구 경기 중에 관중이 수비를 방해할 경우 어떤 규정이 적용되나요?,"[공을 잡으면 정규의 포구가 됩니다., 정규의 포구]"
9,a43cac50-414e-4d66-80ce-3c882dfa1bd9,[[494d83d0-761c-4e71-ae71-de73106e9bda]],[[플레이 시도 시 슬라이딩\r\n주자가 더블 플레이 성립을 방해하기 위한 목적으로...,야구에서 '정당한 슬라이딩'이란 무엇인가요?,[야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'은 베이스...


In [24]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from autorag.data.qa.query.llama_gen_query import llama_index_generate_base
from llama_index.core.base.llms.base import BaseLLM
from typing import Dict

prompt = """
당신은 대한민국의 야구 리그인 KBO의 규정, 규칙, 규약에 대해서 대답하는 시스템의 관리자입니다.
KBO에 관한 질문에 대답해주는 시스템을 위해, 사용자들의 예상 질문을 추출해야 합니다.
주어지는 문서는 2024 KBO리그 규정, 규칙과 규약에 관한 문서입니다.
문서의 내용만을 이용해서 예상되는 사용자들의 질문을 적으세요.
'본 규칙 5.02(c)' 처럼 구체적인 규칙 명을 언급하지 마시오.
단, 반드시 하나의 질문만을 생성하세요.
"""

async def custom_query_gen(
		row: Dict,
		llm: BaseLLM,
		lang: str = "en",
) -> Dict:
	CUSTOM_PROMPT = {
		"ko": [
			ChatMessage(
				role=MessageRole.SYSTEM,
				content=prompt,
			)
		]
	}

	return await llama_index_generate_base(
		row, llm, CUSTOM_PROMPT[lang]
	)

In [25]:
third_qa = initial_qa.batch_apply(
	custom_query_gen,
	llm=llm,
	lang="ko",
)

[01/30/25 14:05:40] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=960880;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=471866;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 14:05:41] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=866656;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=143238;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=478135;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=738203;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=482074;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=444024;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=355879;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=77365;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=164977;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=543643;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=699418;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=717867;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 14:05:42] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=508371;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=902240;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 14:05:43] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=241125;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=986071;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/30/25 14:05:45] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=826012;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=216913;file:///Users/kimbwook/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [26]:
third_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,6d1ec768-e872-44fe-9080-6a4b8828da09,[[2ea156c5-99b4-4338-9306-d139abe8b44e]],[[5.09 아웃\r\n․ 58 ․\r\n다 빨리 2루에 도달했고 타자주자는 1루에...,포스 플레이가 아닌 태그 플레이에서 주자가 제3아웃 되기 전에 본루에 들어간 주자의...,"[포스 상태가 해제되는 경우는 태그 플레이가 발생할 때입니다., 타자가 주자가 됨에..."
1,f99ba0f3-b2da-4cdb-98fc-380fbb2f5e17,[[a389cfdf-361a-45b7-bf77-91ab137e8392]],"[[본 규칙 5.02(c)의 적용을 모면하거나 회피하려고 시도한다면, \r\n심판은...",규칙 5.02(c)를 회피하려는 행위에 대해 어떤 벌칙이 부과되나요?,"[규칙 5.02(c)를 모면하거나 회피하려고 시도하는 경우, 수비팀이 해당 규칙을 ..."
2,f3afe648-ccdd-414b-9dd7-8c054c031bc3,[[2bed192d-9591-405c-a39f-025caee6a7e4]],[[다음 연도 신인 드래프트에서 동일 라운드 종료 후 추가 보\r\n상 지명을 실시...,"어떤 경우에 신인 드래프트 참가 신청서를 제출했지만, 지명된 후 외국 프로구단과 계...",[KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO...
3,50aa50bd-ef0d-47a8-bd18-0b8c7eddd253,[[3d329be3-89db-4e3b-bb83-6ac1d6eb23e6]],"[[출장정지 제재를 받은 자는 제재 종료일까지 KBO 정규시즌, \r\n포스트시즌,...",출장정지 제재를 받은 선수는 어떤 경기에 출장할 수 없나요?,"[출장정지 제재를 받은 자는 KBO 정규시즌, 포스트시즌, 퓨처스리그 경기에 출장할..."
4,d6fd53eb-42ee-4835-83eb-72afac55bf51,[[96db994a-b5f5-49c4-8fff-e6668148da3f]],[[치수를 잴 때는 계\r\n측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉...,어떻게 치수를 측정해야 하나요?,"[계측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉시켜야 합니다., 글러브의..."
5,0bcf8d62-9497-4f5a-a4fa-8626b9b5018b,[[4662da6c-ae20-4889-8289-0fd53b00cc76]],[[쳤다. 땅볼을 받은 \r\n1루수는 1루주자를 포스 아웃시키려고 2루에 송구하였...,"1루수가 1루주자를 포스 아웃시키려다가 투수와 충돌하여 업스트럭션을 선고받았을 때,...","[심판이 투수의 주루방해를 인정하면 ""업스트럭션""을 선고하고 볼을 데드로 하며, 이..."
6,edb95b03-e5aa-4ce7-9307-08505e3c377f,[[07090a61-45c2-4280-b2fb-b3e4647f0e88]],"[[고의낙구 하더라도 5.09⒜⑿의 규정에 관계없이 볼 인 플레이며, \r\n인필드...",인필드 플라이 동안 수비방해가 선언되면 어떤 상황이 벌어지나요?,[인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 인...
7,9e337798-8cb6-4e56-8ebe-a7196abd3888,[[6ebf10d4-1abe-4d0c-b0fb-cc5c0b77eb6d]],"[[시상 \r\n투수 부문 - 평균자책점상, 승리상\r\n타자 부문 - 타율상, 홈...",어떤 기준에 따라 투수와 타자 부문의 시상이 북부와 남부리그로 나누어지나요?,"[야구 시상에서 투수 부문은 평균자책점상과 승리상이 시상되며, 타자 부문은 타율상,..."
8,6a16546d-d5cf-491c-abd7-0df5ff9a5c22,[[3d0914ba-78e8-4572-b4bb-fc6695c94130]],[[설\r\n사 관중이 그 플레이를 방해하더라도 수비방해의 이득을 얻을 수는 없\r...,선수가 덕아웃 안으로 빠지지 않으려고 안쪽에 있는 선수의 신체에 기대하여 플라이 볼...,"[공을 잡으면 정규의 포구가 됩니다., 정규의 포구]"
9,a43cac50-414e-4d66-80ce-3c882dfa1bd9,[[494d83d0-761c-4e71-ae71-de73106e9bda]],[[플레이 시도 시 슬라이딩\r\n주자가 더블 플레이 성립을 방해하기 위한 목적으로...,주자가 더블 플레이 성립을 방해하기 위해 '정당한 슬라이딩'이란 무엇인가요?,[야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'은 베이스...


프롬프트 말을 제대로 듣지 않습니다..

보통 이런 경우에는 조금 더 좋은 모델을 사용하거나 사람이 직접 질문을 편집해주어야 합니다.

이번 실습에서는 거기까지는 다루지는 않겠습니다.

실습의 편의(=가격)를 위해 gpt-3.5-turbo를 사용하였으나 실제 질문 생성에는 더 좋은 LLM 모델을 사용하시길 바랍니다!

강의 촬영일 기준(25.02.06)으로 Deepseek가 굉장히 핫한데, 한국어로 잘 튜닝된 모델을 사용한다면 상당히 저렴하게 좋은 질문 생성을 할 수 있을 것으로 기대됩니다!!

Deepseek를 이용한 실습도 빠르게 준비해봐야겠습니다!
llm만 바꿔주면 되기 때문에, 굉장히 쉽게 사용하실 수 있습니다.

#### 결과 저장
결과물은 굉장히 밉지만... 그래도 열심히 돈 들여 만들었으니 일단 저장해둡니다

In [27]:
human_qa_path = os.path.join(root_dir, 'autorag_project', 'final', 'qa', 'human_qa.parquet')
third_qa.data.to_parquet(human_qa_path)

# 3. 이미 FaQ가 있는 경우
3개의 FaQ를 이용해서 만들어보겠습니다 !

qid, query, generation_gt, retrieval_gt를 만들어서 넣어줘야 합니다

FaQ가 있으니, query와 generation_gt는 이미 있습니다.
qid와 retrieval_gt는 직접 만들어주셔야 합니다.

In [62]:
question = [
    "구단이 보유할 수 있는 선수는 최대 몇 명 인가요?", # 리그 규약 38페이지 24조
    "송구가 심판 맞고 굴절되면 어떻게 되나요? 아웃인가요?", # 야구 규칙 88페이지 ⒡ 감독과 심판원의 방해
    "포스트시즌 경기 입장료는 구단마다 다르던데, 구단 재량인가요?",    # 리그 규정 39페이지 제48조 입장요금(KBO 포스트시즌 경기)
]

In [63]:
answer = [
    "한 구단이 보유할 수 있는 선수의 정원은 보류선수를 포함하여 최대 65명으로 하며 보류선수는 소속선수에 포함된다.",
    "송구가 우연히 베이스 코치에게 닿거나, 투구 또는 송구가 심판원에게 닿았더라도 볼 인 플레이다. 그러나 베이스 코치가 고의로 송구를 방해하였을 경우 주자는 아웃된다.",
    "구단 재량이다. KBO 포스트시즌 경기의 입장요금은 별도로 정한다.",
]

In [64]:
from uuid import uuid4

qid = [str(uuid4()) for _ in range(len(question))]

In [65]:
generation_gt = [[a] for a in answer]

In [66]:
generation_gt

[['한 구단이 보유할 수 있는 선수의 정원은 보류선수를 포함하여 최대 65명으로 하며 보류선수는 소속선수에 포함된다.'],
 ['송구가 우연히 베이스 코치에게 닿거나, 투구 또는 송구가 심판원에게 닿았더라도 볼 인 플레이다. 그러나 베이스 코치가 고의로 송구를 방해하였을 경우 주자는 아웃된다.'],
 ['구단 재량이다. KBO 포스트시즌 경기의 입장요금은 별도로 정한다.']]

이제 retrieval_gt 만 남았습니다

retrieval_gt를 직접 만드는 방법에는 크게 3가지가 있습니다.

1. 직접 찾기
2. 검색 시스템을 만들어 조금 쉽게 찾아보기

저희는 2번 방법을 사용해 만들어보겠습니다!

### BM25를 이용해 retrieval_gt 매핑 검색 시스템 만들기

In [53]:
corpus_dir = os.path.join(root_dir, 'autorag_project', 'final', 'chunk', '0.parquet')
corpus = pd.read_parquet(corpus_dir)
corpus

,doc_id,contents,path,start_end_idx,metadata
0,c7036095-887a-4082-885e-14209c68cb0f,<h1 id='0' style='font-size:18px'>2024년 KBO 규약...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 575]","{'last_modified_datetime': '2025-01-27', 'next..."
1,fc96bdca-c580-4f27-ab43-dfbcdbda99fb,고용할 수 있음</td></tr><tr><td>제44조 [선수계약의 승인]</td>...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[546, 1071]","{'last_modified_datetime': '2025-01-27', 'next..."
2,18e02344-dc14-4924-b38c-9573bd437fcb,<h1 id='0' style='font-size:20px'>제4장 연 고 권</h...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 619]","{'last_modified_datetime': '2025-01-27', 'next..."
3,fd7ba604-3428-4b9b-8dea-8565b394cb34,히어로즈</td><td>서울특별시</td></tr><tr><td>NC 다이노스</t...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[621, 1277]","{'last_modified_datetime': '2025-01-27', 'next..."
4,ef1e83f5-7a4d-419b-b469-f4c4eb53b101,<table id='0' style='font-size:14px'><tr><td>구...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 516]","{'last_modified_datetime': '2025-01-27', 'next..."
...,...,...,...,...,...
856,bb36c5db-a016-404f-ad42-2e2c1eab11e4,그림 8. 정식경기 \r\n<예1>\r\n이닝 1 2 3 4 5 6 7 8 9 \r...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 438]","{'last_modified_datetime': '2025-01-27', 'next..."
857,167952f2-5c16-461c-b139-f7b7d8cf9f40,갑 1 0 0 1 0 0 1 |3\r\n을 0 0 1 0 0 0 0+∝ |1,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[416, 455]","{'last_modified_datetime': '2025-01-27', 'next..."
858,621dcf2e-3fdd-4b8e-8514-b94353524da3,그림 9. 정식경기\r\n<예8>\r\n이닝 1 2 3 4 5 6 7 8 9 \r\...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 447]","{'last_modified_datetime': '2025-01-27', 'next..."
859,006c3812-75b3-4d24-a703-fa261cd59417,3-2 승 \r\n<예14>\r\n이닝 1 2 3 4 5 6 7 8 9 \r\n갑 ...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[429, 526]","{'last_modified_datetime': '2025-01-27', 'next..."


In [54]:
sample_texts = corpus['contents'].tolist()
doc_ids = corpus['doc_id'].tolist()

In [55]:
from langchain_core.documents import Document

docs = [Document(page_content=sample_text, metadata={"doc_id": id}) for sample_text, id in zip(sample_texts, doc_ids)]

In [56]:
# 테디노트님의 라이브러리 활용
# https://wikidocs.net/251980
from langchain_teddynote.retrievers import KiwiBM25Retriever

kiwi = KiwiBM25Retriever.from_documents(docs)

In [57]:
def pretty_print(docs):
    for doc in docs:
        print(f"id: {doc.metadata['doc_id']}\n")
        print(f"content: {doc.page_content}\n\n")

In [58]:
pretty_print(kiwi.invoke(answer[0]))

id: 04a6a16a-728d-4e33-9a15-9d751e74f8aa

content: 38
및 경기력을 발휘함에 지장이 없는 범위에서만 허용되며, 이를 위
하여 구단의 사전 승인을 얻어야 한다.
[2021.10.26 개정]
제24조 [선수정원] ① 한 구단이 보유할 수 있는 선수의 정원은 
보류선수를 포함하여 최대 65명으로 하며 보류선수는 소속선수에 
포함된다.
 ② 군에 입대한 선수는 제1항의 선수정원에 산입하지 아니하며 군
보류선수의 기간은 군입대일로부터 전역일까지로 한다.
[1996.12.28 ➜ 2004.12.7 ➜ 2012.1.10 개정]
※ 제44조 [선수계약의 승인]
제25조 [현역선수] ① 선수가 KBO 정규시즌에 출장하기 위하여는 
KBO에 현역선수로 등록하여야 한다. 
② 총재는 선수가 제1항에 따라 현역선수로 등록하는 경우 이를 
공시하여야 한다. 
제26조 [등록의 시기] ① KBO 정규시즌의 개막 경기부터 출장하
고자 하는 선수는 KBO 정규시즌 개막 예정일의 1일 전 15:00까지 
현역선수 등록을


id: 7da77755-5e6c-4ab5-94f5-bcf8640c5dcb

content: 명단
을 공시한다.
[1996.12.26 ➜ 2012.1.10 ➜ 2014.1.14 개정]
제57조 [보류선수의 정원] 보류선수는 각 구단당 최대 63명을 초
과할 수 없다. 다만, 군보류선수, 육성보류선수는 보류선수의 정원
에 포함되지 아니한다.
제58조 [보류의 효력] ① 보류선수는 제59조의 보류기간 동안 보
류권을 갖는 구단 이외의 다른 구단을 위하여 경기, 훈련 등 야구 
활동을 하거나 다른 구단과 선수계약 체결을 위한 교섭을 할 수 
없다.
② 제1항에도 불구하고 보류선수는 보류권을 갖는 구단의 승인을 
얻어 다른 구단의 합동훈련에 참가할 수 있다. 이 경우 훈련 참가


id: f674a9ce-216c-452d-a835-77542016bbf3

content: 96
공시해야 한다. 
⑥ KBO는 2차 드래프트 다음 날 지명 결과

첫 번째 gt : 04a6a16a-728d-4e33-9a15-9d751e74f8aa

In [59]:
pretty_print(kiwi.invoke(answer[1]))

id: 243d2bb6-81e2-45ba-aca3-483f698ad791

content: 6.01 방해, 업스트럭션
․ 88 ․
⒡ 감독과 심판원의 방해
송구가 우연히 베이스 코치에게 닿거나, 투구 또는 송구가 심판원
에게 닿았더라도 볼 인 플레이다. 그러나 베이스 코치가 고의로 송
구를 방해하였을 경우 주자는 아웃된다. 
[원주] 심판원의 방해 ① 도루를 저지하려는 포수의 송구동작을 주심이 
방해하였을 경우 ② 타구가 야수(투수 제외)를 통과하기 전에 페
어지역에서 심판원에게 닿았을 경우
⒢ 스퀴즈 플레이 또는 도루를 통한 방해
3루주자가 스퀴즈 플레이 또는 도루를 통해 득점하려고 할 때 포수
나 다른 야수가 공을 갖지 않은 채 본루 위 또는 그 앞으로 나오거
나 타자나 타자의 배트를 건드렸을 경우 투수에게 보크를 선고하고 
타자는 인터피어(타격방해)에 의해 1루가 주어진다. 이때는 볼 데
드가 된다.


id: 76df8127-887c-40c8-aa6e-4e53d379c2a9

content: 5.09 아웃
․ 52 ․
[원주] 배트의 부러진 일부분이 페어지역에서 타구에 맞았거나 주자 또
는 야수에게 맞았을 때는 플레이는 그대로 계속되고 방해는 선
언되지 않는다. 타구가 파울지역에서 배트의 부러진 부분에 맞
았을 때는 파울 볼이다. 배트 전체가 페어지역으로 날아가 플레
이를 하려는 야수(타구 처리는 물론 송구도 포함)를 방해하였을 
때는 고의 여부에 관계없이 방해가 선언된다. 타격용 안전모(헬
멧)에 우연히 타구 또는 송구가 맞았을 때는 볼 인 플레이 상태
가 계속된다. 타구가 파울지역에서 타격용 안전모 및 땅이 아닌 
이물질에 닿았을 때는 파울 볼이며, 볼 데드가 된다. 주자가 안
전모를 떨어뜨리거나 공에 던져 타구 또는 송구를 방해하려는 
뜻이 명백하다고 심판원이 판단하였을 경우 그 주자는 아웃이 
되고, 볼 데드가 되어


id: 66e029ea-8a44-49d3-b0a6-ad5a89a00002

content: 참조) 
방해가 고의적일 때는 볼 데

두 번째 gt: 243d2bb6-81e2-45ba-aca3-483f698ad791

In [60]:
pretty_print(kiwi.invoke(answer[2]))

id: b8464983-dbdc-451e-a4a9-72b146508cb3

content: 39
제48조 입장요금(KBO 포스트시즌 경기) 
KBO 포스트시즌 경기의 입장요금은 별도로 정한다.


id: b4993478-f2aa-4e7b-a66d-483be39773e5

content: 31
제33조 경기
1. 모든 경기진행은 KBO가 관장한다.
2. 출장자격은 KBO 한국시리즈에 준한다.
3. 구장 상태, 일기, 기타 사유로 인한 경기개시 여부는 총재가 
이를 결정한다.
4. 입장요금은 KBO 포스트시즌 경기요금에 준한다.


id: b36de4ec-90bd-4276-8378-1731f882c81d

content: 35
제41조 경기 
1. 모든 경기진행은 KBO가 관장한다. 
2. 출장자격은 KBO 한국시리즈에 준한다. 
3. 구장 상태, 일기, 기타 사유로 인한 경기개시 여부는 총재가 
이를 결정한다. 
4. 입장요금은 KBO 포스트시즌 경기요금에 준한다.


id: ca5888b1-0c03-49f8-b592-3aaf2b95cf0d

content: 33
제37조 경기 
1. 모든 경기진행은 KBO가 관장한다. 
2. 출장자격은 KBO 한국시리즈에 준한다. 
3. 구장 상태, 일기, 기타 사유로 인한 경기개시 여부는 총재가 
이를 결정한다. 
4. 입장요금은 KBO 포스트시즌 경기요금에 준한다.




세 번째 gt: b8464983-dbdc-451e-a4a9-72b146508cb3

In [67]:
retrieval_gt = [
    [["04a6a16a-728d-4e33-9a15-9d751e74f8aa"]],
    [["243d2bb6-81e2-45ba-aca3-483f698ad791"]],
    [["b8464983-dbdc-451e-a4a9-72b146508cb3"]],
]

In [68]:
faq_qa = pd.DataFrame({
    "qid": qid,
    "query": question,
    "generation_gt": generation_gt,
    "retrieval_gt": retrieval_gt,
})

In [69]:
faq_qa

,qid,query,generation_gt,retrieval_gt
0,48ae161a-3bd7-4de4-8714-28f0c800e702,구단이 보유할 수 있는 선수는 최대 몇 명 인가요?,[한 구단이 보유할 수 있는 선수의 정원은 보류선수를 포함하여 최대 65명으로 하며...,[[04a6a16a-728d-4e33-9a15-9d751e74f8aa]]
1,fc685c38-63d5-4b09-bab4-a8e1683a59b1,송구가 심판 맞고 굴절되면 어떻게 되나요? 아웃인가요?,"[송구가 우연히 베이스 코치에게 닿거나, 투구 또는 송구가 심판원에게 닿았더라도 볼...",[[243d2bb6-81e2-45ba-aca3-483f698ad791]]
2,4befbd36-59f9-4408-a401-74e3a8518538,"포스트시즌 경기 입장료는 구단마다 다르던데, 구단 재량인가요?",[구단 재량이다. KBO 포스트시즌 경기의 입장요금은 별도로 정한다.],[[b8464983-dbdc-451e-a4a9-72b146508cb3]]


In [70]:
faq_qa_path = os.path.join(root_dir, 'autorag_project', 'final', 'qa', 'faq_qa.parquet')
faq_qa.to_parquet(faq_qa_path)

# QA 데이터 셋으로 합치기

In [71]:
dataframes = [initial_qa.data, third_qa.data, faq_qa]
qa = pd.concat(dataframes, ignore_index=True)
qa

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,6d1ec768-e872-44fe-9080-6a4b8828da09,[[2ea156c5-99b4-4338-9306-d139abe8b44e]],[[5.09 아웃\r\n․ 58 ․\r\n다 빨리 2루에 도달했고 타자주자는 1루에...,포스 상태가 해제되는 경우는 무엇입니까?,"[포스 상태가 해제되는 경우는 태그 플레이가 발생할 때입니다., 타자가 주자가 됨에..."
1,f99ba0f3-b2da-4cdb-98fc-380fbb2f5e17,[[a389cfdf-361a-45b7-bf77-91ab137e8392]],"[[본 규칙 5.02(c)의 적용을 모면하거나 회피하려고 시도한다면, \r\n심판은...",규칙 5.02(c)를 어떻게 모면하거나 회피하려고 시도하는 경우 어떤 벌칙이 부과되나요?,"[규칙 5.02(c)를 모면하거나 회피하려고 시도하는 경우, 수비팀이 해당 규칙을 ..."
2,f3afe648-ccdd-414b-9dd7-8c054c031bc3,[[2bed192d-9591-405c-a39f-025caee6a7e4]],[[다음 연도 신인 드래프트에서 동일 라운드 종료 후 추가 보\r\n상 지명을 실시...,KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO ...,[KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO...
3,50aa50bd-ef0d-47a8-bd18-0b8c7eddd253,[[3d329be3-89db-4e3b-bb83-6ac1d6eb23e6]],"[[출장정지 제재를 받은 자는 제재 종료일까지 KBO 정규시즌, \r\n포스트시즌,...",출장정지 제재를 받은 자는 어떤 경기에 출장할 수 없나요?,"[출장정지 제재를 받은 자는 KBO 정규시즌, 포스트시즌, 퓨처스리그 경기에 출장할..."
4,d6fd53eb-42ee-4835-83eb-72afac55bf51,[[96db994a-b5f5-49c4-8fff-e6668148da3f]],[[치수를 잴 때는 계\r\n측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉...,치수를 잴 때 어디에 계측기나 줄자를 접촉시켜야 하나요?,"[계측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉시켜야 합니다., 글러브의..."
5,0bcf8d62-9497-4f5a-a4fa-8626b9b5018b,[[4662da6c-ae20-4889-8289-0fd53b00cc76]],[[쳤다. 땅볼을 받은 \r\n1루수는 1루주자를 포스 아웃시키려고 2루에 송구하였...,"1루수가 1루주자를 포스 아웃시키려고 2루에 송구했을 때, 심판이 투수의 주루방해를...","[심판이 투수의 주루방해를 인정하면 ""업스트럭션""을 선고하고 볼을 데드로 하며, 이..."
6,edb95b03-e5aa-4ce7-9307-08505e3c377f,[[07090a61-45c2-4280-b2fb-b3e4647f0e88]],"[[고의낙구 하더라도 5.09⒜⑿의 규정에 관계없이 볼 인 플레이며, \r\n인필드...",인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 어떤...,[인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 인...
7,9e337798-8cb6-4e56-8ebe-a7196abd3888,[[6ebf10d4-1abe-4d0c-b0fb-cc5c0b77eb6d]],"[[시상 \r\n투수 부문 - 평균자책점상, 승리상\r\n타자 부문 - 타율상, 홈...","야구 시상에서 투수 부문과 타자 부문으로 나누어 시상하는데, 어떤 상이 시상되는가?","[야구 시상에서 투수 부문은 평균자책점상과 승리상이 시상되며, 타자 부문은 타율상,..."
8,6a16546d-d5cf-491c-abd7-0df5ff9a5c22,[[3d0914ba-78e8-4572-b4bb-fc6695c94130]],[[설\r\n사 관중이 그 플레이를 방해하더라도 수비방해의 이득을 얻을 수는 없\r...,야구에서 덕아웃 부근에서 플라이 볼을 잡으려는 야수가 덕아웃 안으로 빠지지 않으려고...,"[공을 잡으면 정규의 포구가 됩니다., 정규의 포구]"
9,a43cac50-414e-4d66-80ce-3c882dfa1bd9,[[494d83d0-761c-4e71-ae71-de73106e9bda]],[[플레이 시도 시 슬라이딩\r\n주자가 더블 플레이 성립을 방해하기 위한 목적으로...,야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'이란 무엇입니까?,[야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'은 베이스...


retrieval_gt_contents는 지워줘도 상관 없으니 지워줍니다!

In [72]:
qa = qa.drop(columns=["retrieval_gt_contents"])
qa

,qid,retrieval_gt,query,generation_gt
0,6d1ec768-e872-44fe-9080-6a4b8828da09,[[2ea156c5-99b4-4338-9306-d139abe8b44e]],포스 상태가 해제되는 경우는 무엇입니까?,"[포스 상태가 해제되는 경우는 태그 플레이가 발생할 때입니다., 타자가 주자가 됨에..."
1,f99ba0f3-b2da-4cdb-98fc-380fbb2f5e17,[[a389cfdf-361a-45b7-bf77-91ab137e8392]],규칙 5.02(c)를 어떻게 모면하거나 회피하려고 시도하는 경우 어떤 벌칙이 부과되나요?,"[규칙 5.02(c)를 모면하거나 회피하려고 시도하는 경우, 수비팀이 해당 규칙을 ..."
2,f3afe648-ccdd-414b-9dd7-8c054c031bc3,[[2bed192d-9591-405c-a39f-025caee6a7e4]],KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO ...,[KBO 신인 드래프트에서 선수가 외국 프로구단과 계약 협상을 진행한 후에도 KBO...
3,50aa50bd-ef0d-47a8-bd18-0b8c7eddd253,[[3d329be3-89db-4e3b-bb83-6ac1d6eb23e6]],출장정지 제재를 받은 자는 어떤 경기에 출장할 수 없나요?,"[출장정지 제재를 받은 자는 KBO 정규시즌, 포스트시즌, 퓨처스리그 경기에 출장할..."
4,d6fd53eb-42ee-4835-83eb-72afac55bf51,[[96db994a-b5f5-49c4-8fff-e6668148da3f]],치수를 잴 때 어디에 계측기나 줄자를 접촉시켜야 하나요?,"[계측기나 줄자를 글러브의 앞쪽 또는 공을 잡는 쪽에 접촉시켜야 합니다., 글러브의..."
5,0bcf8d62-9497-4f5a-a4fa-8626b9b5018b,[[4662da6c-ae20-4889-8289-0fd53b00cc76]],"1루수가 1루주자를 포스 아웃시키려고 2루에 송구했을 때, 심판이 투수의 주루방해를...","[심판이 투수의 주루방해를 인정하면 ""업스트럭션""을 선고하고 볼을 데드로 하며, 이..."
6,edb95b03-e5aa-4ce7-9307-08505e3c377f,[[07090a61-45c2-4280-b2fb-b3e4647f0e88]],인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 어떤...,[인필드 플라이 동안 수비방해가 선언되면 타구가 페어 / 파울로 결정될 때까지는 인...
7,9e337798-8cb6-4e56-8ebe-a7196abd3888,[[6ebf10d4-1abe-4d0c-b0fb-cc5c0b77eb6d]],"야구 시상에서 투수 부문과 타자 부문으로 나누어 시상하는데, 어떤 상이 시상되는가?","[야구 시상에서 투수 부문은 평균자책점상과 승리상이 시상되며, 타자 부문은 타율상,..."
8,6a16546d-d5cf-491c-abd7-0df5ff9a5c22,[[3d0914ba-78e8-4572-b4bb-fc6695c94130]],야구에서 덕아웃 부근에서 플라이 볼을 잡으려는 야수가 덕아웃 안으로 빠지지 않으려고...,"[공을 잡으면 정규의 포구가 됩니다., 정규의 포구]"
9,a43cac50-414e-4d66-80ce-3c882dfa1bd9,[[494d83d0-761c-4e71-ae71-de73106e9bda]],야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'이란 무엇입니까?,[야구에서 주자가 더블 플레이을 방해하기 위한 목적으로 '정당한 슬라이딩'은 베이스...


In [73]:
qa_path = os.path.join(root_dir, 'autorag_project', 'final', 'qa', 'qa.parquet')
qa.to_parquet(qa_path)